In [5]:
from pia_bench.bench import PiaBenchMark
from dotenv import load_dotenv
import os
load_dotenv()

access_token = os.getenv("ACCESS_TOKEN")
model_name = "T2V_CLIP4CLIP_MSRVTT"
model_name = "T2V_CLIP4Clip_KTT-UCFCrime_CW-pMSRVTT_20241120"
benchmark_path = "assets/PIA"
cfg_target_path= "topk.json"

pia_benchmark = PiaBenchMark(benchmark_path ,model_name=model_name, cfg_target_path= cfg_target_path , token=access_token )
pia_benchmark.preprocess_structure()
# pia_benchmark.preprocess_label_to_csv()  
print("Categories identified:", pia_benchmark.categories)

Folder preprocessing completed.
Categories identified: []


In [ ]:
pia_benchmark.extract_visual_vector()

In [2]:
from pia_bench.event_alarm import EventDetector
detector = EventDetector(config_path=cfg_target_path, model_name=model_name , token=pia_benchmark.token)
detector.process_and_save_predictions(pia_benchmark.vector_video_path, 
                                      pia_benchmark.dataset_path, 
                                      pia_benchmark.alram_path)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Processing videos: 100%|██████████| 94/94 [01:45<00:00,  1.12s/it]


In [4]:
from pia_bench.metric import MetricsEvaluator
metric = MetricsEvaluator(pred_dir=pia_benchmark.alram_path, 
                          label_dir=pia_benchmark.dataset_path, 
                          save_dir=pia_benchmark.metric_path)
metric.evaluate()


Category-wise Average Metrics:

fire:
falldown_f1: 0.000
falldown_accuracy: 0.745
falldown_precision: 0.000
falldown_recall: 0.000
falldown_specificity: 0.745
falldown_tp: 0.000
falldown_tn: 1675.184
falldown_fp: 903.026
falldown_fn: 0.000
violence_f1: 0.000
violence_accuracy: 0.915
violence_precision: 0.000
violence_recall: 0.000
violence_specificity: 0.915
violence_tp: 0.000
violence_tn: 2393.789
violence_fp: 184.421
violence_fn: 0.000
fire_f1: 0.811
fire_accuracy: 0.754
fire_precision: 0.753
fire_recall: 0.945
fire_specificity: 0.002
fire_tp: 953.237
fire_tn: 3.947
fire_fp: 1619.447
fire_fn: 1.579

violence:
falldown_f1: 0.043
falldown_accuracy: 0.703
falldown_precision: 0.024
falldown_recall: 0.302
falldown_specificity: 0.701
falldown_tp: 108.867
falldown_tn: 4641.133
falldown_fp: 2463.500
falldown_fn: 69.167
violence_f1: 0.155
violence_accuracy: 0.661
violence_precision: 0.159
violence_recall: 0.266
violence_specificity: 0.779
violence_tp: 313.533
violence_tn: 4831.900
violence_f

{'fire': {'video_name': 'average',
  'falldown_f1': 0.0,
  'falldown_accuracy': 0.7451854056679043,
  'falldown_precision': 0.0,
  'falldown_recall': 0.0,
  'falldown_specificity': 0.7451854056679043,
  'falldown_tp': 0.0,
  'falldown_tn': 1675.1842105263158,
  'falldown_fp': 903.0263157894736,
  'falldown_fn': 0.0,
  'violence_f1': 0.0,
  'violence_accuracy': 0.9153883892338825,
  'violence_precision': 0.0,
  'violence_recall': 0.0,
  'violence_specificity': 0.9153883892338825,
  'violence_tp': 0.0,
  'violence_tn': 2393.7894736842104,
  'violence_fp': 184.42105263157896,
  'violence_fn': 0.0,
  'fire_f1': 0.8108560072968086,
  'fire_accuracy': 0.7537196673581136,
  'fire_precision': 0.7528824760479286,
  'fire_recall': 0.9452853770309679,
  'fire_specificity': 0.002236783678613687,
  'fire_tp': 953.2368421052631,
  'fire_tn': 3.9473684210526314,
  'fire_fp': 1619.4473684210527,
  'fire_fn': 1.5789473684210527},
 'violence': {'video_name': 'average',
  'falldown_f1': 0.042545154996438